This Notebook serves as a tool to get daily METARS from any available part of the world. It is very usefull to know some of the country codes, but you can read them in https://mesonet.agron.iastate.edu/sites/networks.php?.

There are two main functions. First

```
fetch_and_process_csv()
```
This function works if you know what stations you know. It accepts ICAO codes.

If you do not know, you know only country and the location, use:

```
find_closest_locations()
```
Generates rows of the closest locations within the area.
Then, you can retrieve the ICAO indicators.


In [16]:
import requests
import pandas as pd
from io import StringIO

def build_URL_metars(st_year, st_mon, st_day, en_year, en_mon, en_day, stations, data_li = ['sknt','vsby'], latlon = 'yes'):
    """
    Builds URL to the METARS from NCAR based on

    Parameters:
    - st_year, st_mon, st_day: integers of starting year, month, day.
    - en_year, en_mon, en_day: integers of ending year, month, day.
    - stations (list): List of four letters ICAO codes, e.g. ['LKPR', 'EDDM'],
    - data_li (list): list of parameters that should be in separate csv columns, default ['sknt','vsby'].
    - latlon ('yes' or 'no'): if latitude and longitude should be included in the csv
    """
    url1 = 'https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?'
    stations_str = ''.join(['station='+st + '&' for st in stations])
    data_str = ''.join('data='+dat + '&' for dat in data_li)

    st_day_str = f'year1={str(st_year)}&month1={str(st_mon)}&day1={str(st_day)}&'
    en_day_str = f'year2={str(st_year)}&month2={str(st_mon)}&day2={str(st_day)}&'

    url2 = f'tz=Etc%2FUTC&format=onlycomma&latlon={latlon}&elev=no&missing=M&trace=T&direct=no&report_type=3&report_type=4'
    url_final = f'{url1}{stations_str}data=metar&{data_str}{st_day_str}{en_day_str}{url2}'
    return url_final

def fetch_and_process_csv(url, print_data=False, save_to_file=None):
    """
    Fetches CSV METARS or stations and other data from the specified URL, processes it using pandas,
    and either prints it to the console or saves it to a CSV file.

    Parameters:
    - url (str): The URL containing the CSV metar data.
    - print_data (bool): If True, print the CSV data to the console.
    - save_to_file (str or None): If provided, save the CSV data to the specified file.
    """
    try:
        # Fetch CSV data from the URL
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for HTTP errors

        # Process CSV data using pandas
        csv_data = pd.read_csv(StringIO(response.text))

        # Print to console if specified
        if print_data:
            print(csv_data)

        # Save to a file if specified
        if save_to_file:
            csv_data.to_csv(save_to_file, index=False)
            print(f"CSV data saved to {save_to_file}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")

    return csv_data

In [ ]:
#@title Define parameters and Example usage
st_year = 2023
st_mon = 10
st_day = 1

en_year = 2023
en_mon = 11
en_day = 3

stations = ['LKTB', 'LKNA']

data_li = ['sknt','vsby'] #choose parameters to separate in columns see last cell to find parameters
latlon = 'yes' # 'no' if you do not want to include

#example usage

path = build_URL_metars(st_year, st_mon, st_day, en_year, en_mon, en_day, stations, data_li = ['sknt','vsby'], latlon = 'yes')
fetch_and_process_csv(path)

In [7]:
#@title Generate list of Airports within Network

def generate_asos_df(ASOS):
    """
    Returns dataframe of the airports from the NCAR archive.

    Parameters:
    - ASOS (str): Two letters Asos code, e.g. FR:France, CZ: Czech rep., GB: Great Britain...
    - outp_DataFrame  (bool): If True, print the CSV data to the console.
    - save_to_file (str or None): If provided, save the CSV data to the specified file.
    """
    #asos_network url
    url = f'https://mesonet.agron.iastate.edu/sites/networks.php?network={ASOS}__ASOS&format=csv&nohtml=on'
    df = fetch_and_process_csv(url)
    return df


In [9]:
df = generate_asos_df('FR')
df.head()

,stid,station_name,lat,lon,elev,begints,endts,iem_network
0,LFOI,Abbeville,50.13611,1.83889,77.0,1928-01-01 00:00,2012-02-05 00:00,FR__ASOS
1,LFBA,Agen,44.17222,0.59472,60.0,1972-12-31 00:00,NaN,FR__ASOS
2,LFKJ,Ajaccio,41.93056,8.79278,9.0,1936-06-16 00:00,NaN,FR__ASOS
3,LFAQ,Albert,49.97150,2.69770,111.0,2007-06-06 00:00,2022-11-07 00:00,FR__ASOS
4,LFOF,ALENCON/VALFRAMBERT,48.44556,0.11028,141.0,1972-12-31 00:00,2014-01-30 00:00,FR__ASOS


In [13]:
from geopy.distance import geodesic

def find_closest_locations(df, target_lat, target_lon, num_closest=5):
    """
    Find the N closest rows in the DataFrame to the target coordinates.

    Parameters:
    - df (pd.DataFrame): DataFrame containing location data.
    - target_lat (float): Target latitude.
    - target_lon (float): Target longitude.
    - num_closest (int): Number of closest locations to return.

    Returns:
    - pd.DataFrame: DataFrame with the N closest rows.
    """
    # Create a tuple for the target coordinates
    target_coords = (target_lat, target_lon)

    # Apply the geodesic function to calculate distances for each row in the DataFrame
    distances = df.apply(lambda row: geodesic(target_coords, (row['lat'], row['lon'])).kilometers, axis=1)

    # Find the indices of the N rows with the smallest distances
    closest_indices = distances.nsmallest(num_closest).index

    # Return the DataFrame with the N closest rows
    closest_locations = df.loc[closest_indices]

    return closest_locations


In [15]:
# Example usage:
# Assuming you have a DataFrame df with columns 'lat' and 'lon'
user_lat = 49.17  # Example latitude
user_lon = 16.10  # Example longitude

closest_locations = find_closest_locations(df, user_lat, user_lon, num_closest=5)

closest_locations.head()

,stid,station_name,lat,lon,elev,begints,endts,iem_network
111,LFST,Strasbourg,48.54944,7.64028,154.00000,1931-01-01 00:00,NaN,FR__ASOS
38,LFGA,Colmar,48.10990,7.35900,183.72566,2014-09-12 00:00,2021-08-26 00:00,FR__ASOS
78,LFSB,Mulhouse,47.61417,7.51028,271.00000,1972-12-31 00:00,NaN,FR__ASOS
71,LFJL,Metz,48.97944,6.24306,264.00000,1992-04-08 00:00,NaN,FR__ASOS
80,LFSN,Nancy,48.68778,6.22167,217.00000,1928-01-01 00:00,NaN,FR__ASOS


In [ ]:
#@title Parameters list
'''
station: three or four character site identifier
valid:timestamp of the observation
tmpf: Air Temperature in Fahrenheit, typically @ 2 meters
dwpf: Dew Point Temperature in Fahrenheit, typically @ 2 meters
relh: Relative Humidity in %
drct: Wind Direction in degrees from *true* north
sknt Wind Speed in knots
p01i: One hour precipitation for the period from the observation time to the time of the previous hourly precipitation reset. This varies slightly by site. Values are in inches. This value may or may not contain frozen precipitation melted by some device on the sensor or estimated by some other means. Unfortunately, we do not know of an authoritative database denoting which station has which sensor.
alti: Pressure altimeter in inches
mslp: ea Level Pressure in millibar
vsby: Visibility in miles
gust: Wind Gust in knots
skyc1: Sky Level 1 Coverage
skyc2: Sky Level 2 Coverage
skyc3: Sky Level 3 Coverage
skyc4: Sky Level 4 Coverage
skyl1: Sky Level 1 Altitude in feet
skyl2: Sky Level 2 Altitude in feet
skyl3: Sky Level 3 Altitude in feet
skyl4: Sky Level 4 Altitude in feet
wxcodes: Present Weather Codes (space seperated)
feel: Apparent Temperature (Wind Chill or Heat Index) in Fahrenheit
ice_accretion_1hr: Ice Accretion over 1 Hour (inches)
ice_accretion_3hr: Ice Accretion over 3 Hours (inches)
ice_accretion_6hr: Ice Accretion over 6 Hours (inches)
peak_wind_gust: Peak Wind Gust (from PK WND METAR remark) (knots)
peak_wind_drct: Peak Wind Gust Direction (from PK WND METAR remark) (deg)
peak_wind_time: Peak Wind Gust Time (from PK WND METAR remark)
metar: unprocessed reported observation in METAR format'''